In [1]:
from sklearn import svm, gaussian_process, tree, neighbors

from data_preparation import *
from util import *

np.random.seed(0)

In [2]:
df = load_preprocessed_dataset(remove_duplicates=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1638 entries, 1909 to 768
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   storageRegion        1638 non-null   object        
 1   storageProvider      1638 non-null   object        
 2   functionId           1638 non-null   object        
 3   functionName         1638 non-null   object        
 4   functionType         1638 non-null   object        
 5   RTT                  1638 non-null   float64       
 6   loopCounter          1638 non-null   float64       
 7   maxLoopCounter       1638 non-null   float64       
 8   startTime            1638 non-null   datetime64[ns]
 9   endTime              1638 non-null   datetime64[ns]
 10  upAll                1638 non-null   float64       
 11  downAll              1638 non-null   float64       
 12  numberDownloadFiles  1638 non-null   int64         
 13  sizeDownloadInMB     1638 non-null  

In [3]:
def ablation_input_study(input_cols, model):
    X_train, y_train, groups_train, X_test, y_test, _, df_test = train_test_split_with_criterion(
        lambda x: (x['wfType'] == 'bwa' and x['functionProvider'] == 'AWS'), df, input_cols, "RTT", "kFoldGroupEnc")

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    rmse = np.sqrt(s_m.mean_squared_error(y_test, predictions))
    mae = s_m.mean_absolute_error(y_test, predictions)
    mape = s_m.mean_absolute_percentage_error(y_test, predictions)
    print('%.2f & %.2f & %.2f \\\\' % (rmse, mae, mape))

In [4]:
ablation_input_study(get_function_related_cols(), svm.SVR())

11.88 & 9.18 & 1.99 \\


In [5]:
ablation_input_study(get_function_related_cols() + get_storage_related_cols(), svm.SVR())

6.58 & 5.82 & 0.99 \\


In [6]:
ablation_input_study(get_function_related_cols() + get_storage_related_cols() + get_concurrency_related_cols(),
                     svm.SVR())

6.52 & 5.77 & 0.98 \\


In [7]:
ablation_input_study(
    get_function_related_cols() + get_storage_related_cols() + get_concurrency_related_cols() + get_time_related_cols(),
    svm.SVR())

6.72 & 5.40 & 0.77 \\


In [8]:
ablation_input_study(get_function_related_cols(), neighbors.KNeighborsRegressor())

4.93 & 3.83 & 0.39 \\


In [9]:
ablation_input_study(get_function_related_cols() + get_storage_related_cols(), neighbors.KNeighborsRegressor())

1.73 & 1.31 & 0.16 \\


In [10]:
ablation_input_study(get_function_related_cols() + get_storage_related_cols() + get_concurrency_related_cols(), neighbors.KNeighborsRegressor())

2.55 & 1.52 & 0.14 \\


In [11]:
ablation_input_study(get_function_related_cols() + get_storage_related_cols() + get_concurrency_related_cols() + get_time_related_cols(), neighbors.KNeighborsRegressor())

8.21 & 6.80 & 0.82 \\


In [12]:
ablation_input_study(get_function_related_cols() + get_storage_related_cols() + get_time_related_cols(),
                     svm.SVR())

6.72 & 5.40 & 0.77 \\


In [13]:
ablation_input_study(get_function_related_cols() + get_storage_related_cols() + get_time_related_cols(), neighbors.KNeighborsRegressor())

8.20 & 6.78 & 0.81 \\
